In [27]:
import pandas as pd

In [28]:
# load the first n rows from data/lob/full_lob.csv
df = pd.read_csv('data/lob/full_lob.csv', index_col=0)
df

,Timestamp,Exchange,Order Type,Price,Quantity,Date
0,0.000,Exch0,Bid,NaN,NaN,2025-05-13
1,1.612,Exch0,Bid,1.0,1.0,2025-05-13
2,2.170,Exch0,Bid,1.0,1.0,2025-05-13
3,2.449,Exch0,Bid,2.0,1.0,2025-05-13
4,2.945,Exch0,Bid,3.0,1.0,2025-05-13
...,...,...,...,...,...,...
82477385,30599.945,Exch0,Ask,211.0,3.0,2025-05-06
82477385,30599.945,Exch0,Ask,333.0,3.0,2025-05-06
82477385,30599.945,Exch0,Ask,580.0,5.0,2025-05-06
82477385,30599.945,Exch0,Ask,605.0,3.0,2025-05-06


In [29]:
# Ensure 'Timestamp' is treated as a number
df['Timestamp'] = pd.to_numeric(df['Timestamp'])
df['Price'] = pd.to_numeric(df['Price'])
df['Quantity'] = pd.to_numeric(df['Quantity'])

# Sort the DataFrame by date and Timestamp to ensure it's in chronological order
df.sort_values(by=['Date','Timestamp'], inplace=True)


In [30]:
df = df.head(5000000)

In [31]:
# remove all NaN values from price and quantity
# df = df.dropna(subset=['Price', 'Quantity'])

# Function to calculate best bid and ask
def get_best_bid_ask(sub_df):
    best_bid = sub_df[sub_df['Order Type'] == 'Bid']['Price'].max()
    best_ask = sub_df[sub_df['Order Type'] == 'Ask']['Price'].min()
    return pd.Series([best_bid, best_ask], index=['Best Bid', 'Best Ask'])

# Apply the function to each timestamp and create new columns for best bid and ask
df[['Best Bid', 'Best Ask']] = df.groupby('Timestamp').apply(get_best_bid_ask)

# df = df.dropna()

# Calculate the mid-price
df['Mid Price'] = (df['Best Bid'] + df['Best Ask']) / 2

# Forward fill the NaN values to ensure each row has a bid, ask, and mid-price
df[['Best Bid', 'Best Ask', 'Mid Price']] = df[['Best Bid', 'Best Ask', 'Mid Price']].ffill()

# Shift the mid-prices by 1 to calculate the future mid-price based on the immediate next timestamp
df['Future Mid Price'] = df['Mid Price'].shift(-1)

# # Calculate the percentage change between the current and future mid-prices
# df['Mid Price Movement'] = (df['Future Mid Price'] - df['Mid Price']) / df['Mid Price']

# # Define new thresholds for categorizing the mid-price movement
# threshold_up = 0.005  # 0.5% threshold for up movement
# threshold_down = -0.005  # -0.5% threshold for down movement

# # Categorize the mid-price movement with the new thresholds
# df['Mid Price Movement Category'] = pd.cut(df['Mid Price Movement'], bins=[-float('inf'), threshold_down, threshold_up, float('inf')], labels=['down', 'stationary', 'up'])

# # Drop NaN values that were created by shifting
# df.dropna(subset=['Future Mid Price', 'Mid Price Movement', 'Mid Price Movement Category'], inplace=True)

# Now `df` contains the original data along with the calculated best bid, best ask, mid-price, future mid-price, mid-price movement, and the categorized mid-price movement
df

/var/folders/yw/qmvvfjd916v9cr0f447rhf700000gp/T/ipykernel_4863/394485628.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Best Bid', 'Best Ask']] = df.groupby('Timestamp').apply(get_best_bid_ask)
/var/folders/yw/qmvvfjd916v9cr0f447rhf700000gp/T/ipykernel_4863/394485628.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Best Bid', 'Best Ask']] = df.groupby('Timestamp').apply(get_best_bid_ask)
/var/folders/yw/qmvvfjd916v9cr0f447rhf700000gp/T/ipykernel_4863/394485628.py:16: SettingWithCopyWarn

,Timestamp,Exchange,Order Type,Price,Quantity,Date,Best Bid,Best Ask,Mid Price,Future Mid Price
25781997,0.000,Exch0,Bid,NaN,NaN,2025-01-02,NaN,NaN,NaN,NaN
67020690,0.000,Exch0,Ask,NaN,NaN,2025-01-02,NaN,NaN,NaN,NaN
25781998,0.279,Exch0,Bid,1.0,6.0,2025-01-02,NaN,NaN,NaN,NaN
67020691,0.279,Exch0,Ask,NaN,NaN,2025-01-02,NaN,NaN,NaN,NaN
25781999,1.333,Exch0,Bid,1.0,6.0,2025-01-02,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
26123018,29573.938,Exch0,Bid,292.0,2.0,2025-01-02,NaN,NaN,NaN,NaN
26123018,29573.938,Exch0,Bid,291.0,9.0,2025-01-02,NaN,NaN,NaN,NaN
26123018,29573.938,Exch0,Bid,288.0,3.0,2025-01-02,NaN,NaN,NaN,NaN
26123018,29573.938,Exch0,Bid,287.0,4.0,2025-01-02,NaN,NaN,NaN,NaN


In [32]:
# show value counts for the mid-price movement category
df['Future Mid Price'].value_counts()

Series([], Name: count, dtype: int64)

In [33]:
# pivot on order type
df = df.pivot(index=['Timestamp','Exchange','Date'], columns='Order Type', values=['Price','Quantity'])
df

ValueError: Index contains duplicate entries, cannot reshape